## Overview

This colab explores chains available in Langchain through practical examples.

We build a basic use case of generating the product name and tagline given a description of the product.

This is followed by a more complex case where the tagline is decided based on the target age group of the product.

Design of the flow is provided in the deck available on the [Github repo](https://github.com/InFoCusp/langchain_tutorial/tree/main)

# Installs

In [ ]:
! pip install langchain

# For Google PALM access
! pip install google-generativeai

# For Serp API
! pip install google-search-results

# Imports and General setup

In [ ]:
import os
import json
import time
from pprint import pprint
from typing import Dict, Tuple, List, Union, Optional, Callable, Any

In [ ]:
# Connect to drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# PALM Setup

In [ ]:
# Generate yours @ https://developers.generativeai.google/products/palm
with open('/content/drive/My Drive/LangChain course/google_palm_key.txt', 'r') as f:
  palm_api_key = f.readlines()[0].strip('\n')

os.environ["GOOGLE_API_KEY"] = palm_api_key

In [ ]:
import langchain
from langchain.prompts import PromptTemplate
from langchain.llms.google_palm import GooglePalm

llm = GooglePalm(temperature=0.0, max_output_tokens=256)
llm

GooglePalm(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<module 'google.generativeai' from '/usr/local/lib/python3.10/dist-packages/google/generativeai/__init__.py'>, google_api_key=None, model_name='models/text-bison-001', temperature=0.0, top_p=None, top_k=None, max_output_tokens=256, n=1)

# Chains

## LLMChain

In [ ]:
from langchain.chains import LLMChain

In [ ]:
company = "Audio Mechanics"

product_description = """The perfect headphones for young music lovers.
With their sleek design and powerful sound, these headphones are sure to turn heads wherever you go. The over-ear design provides
superior comfort, even during extended listening sessions, while the noise-canceling technology blocks out distractions so you
can focus on your music."""

### Name generator chain

In [ ]:
prompt = PromptTemplate(
    input_variables=["company", "product_description"],
    template="""What is a good product name for a product with the description `{product_description}` and made by the company {company}?""",
)

name_generator_chain = LLMChain(llm=llm, prompt=prompt,
                                output_key='product_name')

print(
    name_generator_chain.run({
        'company': company,
        'product_description': product_description,
    })
)

Audio Mechanics M1000 Over-Ear Headphones


### Tag-line generator chain

In [ ]:
prompt = PromptTemplate(
    input_variables=["company", "product_description", "product_name"],
    template="""You are a copywriter that writes tag lines. What is a good tag line for
a product called {product_name} with the description `{product_description}` and made by the company {company}?""",
)

tagline_generator_chain = LLMChain(llm=llm, prompt=prompt,
                                   output_key="tag-line")

print(tagline_generator_chain.run({
    'company': company,
    'product_description': product_description,
    'product_name': 'Audio Mechanics M1000 Over-Ear Headphones'
}))

**Audio Mechanics M1000 Over-Ear Headphones: Sleek design, powerful sound, and superior comfort.**


## Sequential Chain

In [ ]:
from langchain.chains import SequentialChain

full_chain = SequentialChain(
    chains=[name_generator_chain, tagline_generator_chain],
    input_variables=["company", "product_description"],
    output_variables=["product_name", "tag-line"],
)

pprint(
    full_chain(
        {
            'company': company,
            'product_description': product_description
        },
        return_only_outputs=True
    )
)

{'product_name': 'Audio Mechanics M1000 Over-Ear Headphones',
 'tag-line': '**Audio Mechanics M1000 Over-Ear Headphones: Sleek design, '
             'powerful sound, and superior comfort.**'}


## Multi-prompt and Router Chain

### Create default chain

In [ ]:
default_template =  """You are a copywriter that writes tag lines. What is a good tag line
given the following info:
{input}"""

prompt = PromptTemplate(template=default_template, input_variables=["input"])
default_chain = LLMChain(llm=llm, prompt=prompt)

### Create audience based tag-line generator chains

In [ ]:
teens_template = """You are a copywriter that writes tag lines for customers in their teens. What is a good tag line
given the following info:
{input}"""

middle_aged_template = """You are a copywriter that writes tag line for customers in their late 30s. What is a good tag line
given the following info:
{input}"""

prompt_infos = [
    {
        "name": "teens",
        "description": "Good for customers in their teens.",
        "prompt_template": teens_template,
    },
    {
        "name": "middle-aged",
        "description": "Good for middle aged customers.",
        "prompt_template": middle_aged_template,
    },
]

destination_chains = {}
for p_info in prompt_infos:

    name = p_info["name"]
    prompt_template = p_info["prompt_template"]

    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)

    destination_chains[name] = chain

### Create router chain

In [ ]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]

destinations_str = "\n".join(destinations)

router_template = f"""Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": {{{{
        "company": string \ name of the company from original input
        "product_name": string \ name of the product from original input
        "product_description": string \ description of the product from original input
    }}}}
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations_str}

<< INPUT >>
company: {{company}}
product_name: {{product_name}}
product_description : {{product_description}}

<< OUTPUT >>"""

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["company", "product_description", "product_name"],
    output_parser=RouterOutputParser(next_inputs_type=dict),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

### Create multi-prompt chain

In [ ]:
from langchain.chains.router import MultiPromptChain

multi_prompt_chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
)

In [ ]:
print(
    multi_prompt_chain.run({
        'company': company,
        'product_description': product_description,
        'product_name': "Audio Mechanics M1000 Over-Ear Headphones"
    })
)

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


**Audio Mechanics M1000 Over-Ear Headphones: Turn your world up to 11.**


### Combine name generator and the multi-prompt tag-line generator

In [ ]:
from langchain.chains import SequentialChain

full_chain = SequentialChain(
    chains=[name_generator_chain, multi_prompt_chain],
    input_variables=["company", "product_description"],
)

### Run

In [ ]:
# Teens example.
print(full_chain.run({
    'company': company,
    'product_description': product_description
}))

In [ ]:
# Teens example.
langchain.debug=True
print(full_chain.run({
    'company': company,
    'product_description': product_description
}))
langchain.debug=False

[chain/start] [1:chain:SequentialChain] Entering Chain run with input:
{
  "company": "Audio Mechanics",
  "product_description": "The perfect headphones for young music lovers. \nWith their sleek design and powerful sound, these headphones are sure to turn heads wherever you go. The over-ear design provides\nsuperior comfort, even during extended listening sessions, while the noise-canceling technology blocks out distractions so you \ncan focus on your music."
}
[chain/start] [1:chain:SequentialChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "company": "Audio Mechanics",
  "product_description": "The perfect headphones for young music lovers. \nWith their sleek design and powerful sound, these headphones are sure to turn heads wherever you go. The over-ear design provides\nsuperior comfort, even during extended listening sessions, while the noise-canceling technology blocks out distractions so you \ncan focus on your music."
}
[llm/start] [1:chain:SequentialChain > 2:chai

In [ ]:
# Middle-aged example.
langchain.debug=True
print(full_chain.run({
    'company': "AntiAge",
    'product_description': """A luxurious anti-aging cream that reduces the appearance of wrinkles and fine lines, leaving skin looking youthful and radiant."""
}))
langchain.debug=False

[chain/start] [1:chain:SequentialChain] Entering Chain run with input:
{
  "company": "AntiAge",
  "product_description": "A luxurious anti-aging cream that reduces the appearance of wrinkles and fine lines, leaving skin looking youthful and radiant."
}
[chain/start] [1:chain:SequentialChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "company": "AntiAge",
  "product_description": "A luxurious anti-aging cream that reduces the appearance of wrinkles and fine lines, leaving skin looking youthful and radiant."
}
[llm/start] [1:chain:SequentialChain > 2:chain:LLMChain > 3:llm:GooglePalm] Entering LLM run with input:
{
  "prompts": [
    "What is a good product name for a product with the description `A luxurious anti-aging cream that reduces the appearance of wrinkles and fine lines, leaving skin looking youthful and radiant.` and made by the company AntiAge?"
  ]
}
[llm/end] [1:chain:SequentialChain > 2:chain:LLMChain > 3:llm:GooglePalm] [335.462ms] Exiting LLM run with output

In [ ]:
# Default example.
langchain.debug=True
print(full_chain.run({
    'company': "TravelComfort",
    'product_description': """A portable luggage scale that is perfect for frequent work travels. It is lightweight and compact, making
it easy to take with you on your next trip. The scale has a large LCD display that makes it easy to read, even in bright conditions."""
}))
langchain.debug=True

[chain/start] [1:chain:SequentialChain] Entering Chain run with input:
{
  "company": "TravelComfort",
  "product_description": "A portable luggage scale that is perfect for frequent work travels. It is lightweight and compact, making \nit easy to take with you on your next trip. The scale has a large LCD display that makes it easy to read, even in bright conditions."
}
[chain/start] [1:chain:SequentialChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "company": "TravelComfort",
  "product_description": "A portable luggage scale that is perfect for frequent work travels. It is lightweight and compact, making \nit easy to take with you on your next trip. The scale has a large LCD display that makes it easy to read, even in bright conditions."
}
[llm/start] [1:chain:SequentialChain > 2:chain:LLMChain > 3:llm:GooglePalm] Entering LLM run with input:
{
  "prompts": [
    "What is a good product name for a product with the description `A portable luggage scale that is perfect for